In [0]:
import numpy as np
import pandas as pd
import nltk
import spacy
import os
import string
import gensim

In [0]:
train = pd.read_csv("/content/train_2.csv")
train = train.dropna()
train.head()

,id,qid1,qid2,question1,question2,is_duplicate,Q1,Q2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"['step', 'step', 'guide', 'invest', 'share', '...","['step', 'step', 'guide', 'invest', 'share', '..."
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"['story', 'kohinoor', 'kohinoor', 'diamond']","['would', 'happen', 'indian', 'government', 's..."
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"['increase', 'speed', 'internet', 'connection'...","['internet', 'speed', 'increased', 'hacking', ..."
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"['mentally', 'lonely', 'solve']","['find', 'remainder', 'math2324math', 'divided..."
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"['one', 'dissolve', 'water', 'quikly', 'sugar'...","['fish', 'would', 'survive', 'salt', 'water']"


In [0]:
!wget -c http://nlp.stanford.edu/data/glove.840B.300d.zip

In [0]:
import zipfile
with zipfile.ZipFile("/content/glove.840B.300d.zip", "r") as zipread:
  zipread.extractall("/content/")
  zipread.close

import gensim
from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec

PATH_TO_GLOVE = "/content/glove.840B.300d.txt"

tmp_file = "/glove.840B.300d.w2v.txt"
glove2word2vec(PATH_TO_GLOVE, tmp_file)
glove = gensim.models.KeyedVectors.load_word2vec_format(tmp_file)

In [0]:
def wmd(s1, s2):
  return -glove.wmdistance(s1, s2)

In [0]:
train["distance"] = train.apply(lambda x: wmd(x.Q1, x.Q2), axis=1)
train.head()

,id,qid1,qid2,question1,question2,is_duplicate,Q1,Q2,distance
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"['step', 'step', 'guide', 'invest', 'share', '...","['step', 'step', 'guide', 'invest', 'share', '...",-0.479549
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"['story', 'kohinoor', 'kohinoor', 'diamond']","['would', 'happen', 'indian', 'government', 's...",-1.440303
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"['increase', 'speed', 'internet', 'connection'...","['internet', 'speed', 'increased', 'hacking', ...",-0.953338
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"['mentally', 'lonely', 'solve']","['find', 'remainder', 'math2324math', 'divided...",-3.211461
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"['one', 'dissolve', 'water', 'quikly', 'sugar'...","['fish', 'would', 'survive', 'salt', 'water']",-1.451814


In [0]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(train[["distance"]], train["is_duplicate"], random_state = 0, test_size=0.2)

In [0]:
import xgboost as xgb
options = {"model_param":
         {"eta": 0.15, 
         "max_depth": 4,
         "n_estimators":50,
         "sub_sample":0.5,
         "scale_pos_weight":1.75,
         "random_state":1,
         "eval_metric": "logloss",
         "objective": "binary:logistic"},
         "num_round":50,
         "test_frac":0.2}

In [0]:
#training model
xgbmatrix = xgb.DMatrix(train_X, train_y)
xgb_model = xgb.train(options["model_param"], xgbmatrix, options["num_round"], verbose_eval=10)
xgb_valmatrix = xgb.DMatrix(val_X)
y_val_pred = xgb_model.predict(xgb_valmatrix)
y_val_pred #just checking

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


array([0.62029463, 0.530533  , 0.3723144 , ..., 0.84838665, 0.57748425,
       0.316866  ], dtype=float32)

In [0]:
y_val_pred_binary = [1 if y>0.5 else 0 for y in y_val_pred]
from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix(val_y, y_val_pred_binary)

array([[30706, 20432],
       [ 9258, 20462]])

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(val_y, y_val_pred_binary)

0.632813079720003